In [1]:
import pandas as pd
import numpy as np
from context_engineering_functions import create_basic_triples

context = create_basic_triples('csgo_clean', save=False)

Finished Basic Context Engineering


In [2]:
context.head()

,MatchId,de_dust2_is_available,de_inferno_is_available,de_mirage_is_available,de_nuke_is_available,de_overpass_is_available,de_train_is_available,de_vertigo_is_available,DecisionTeamId,OtherTeamId,DecisionOrder,X_Action,Y_reward
0,4,1,1,1,1,1,1,1,12,6,1,6,-1.0
1,4,1,1,1,1,1,1,0,6,12,2,4,-1.0
2,4,1,1,1,1,0,1,0,12,6,3,0,0.0
3,4,0,1,1,1,0,1,0,6,12,4,3,1.0
4,4,0,1,1,0,0,1,0,12,6,5,5,-1.0


In [3]:
context = context[context['DecisionOrder'].isin([1, 2, 5, 6])]

In [19]:
context = context[~(context[cols] == 0).all(axis=1)]

In [20]:
context.shape

(10794, 13)

In [5]:
%load_ext autoreload
%autoreload 2

In [6]:
context.reset_index(drop=True, inplace=True)

In [28]:
from bandit import VetoBandit

In [29]:
context.head()

,MatchId,de_dust2_is_available,de_inferno_is_available,de_mirage_is_available,de_nuke_is_available,de_overpass_is_available,de_train_is_available,de_vertigo_is_available,DecisionTeamId,OtherTeamId,DecisionOrder,X_Action,Y_reward
0,4,1,1,1,1,1,1,1,12,6,1,6,-1.0
1,4,1,1,1,1,1,1,0,6,12,2,4,-1.0
2,4,0,1,1,0,0,1,0,12,6,5,5,-1.0
3,4,0,1,1,0,0,0,0,6,12,6,2,-1.0
5,5,1,1,1,1,0,1,1,5,9,2,0,-1.0


In [30]:
# do we need a train test split?

cols = [col for col in context if col.endswith('is_available')]

X = context[cols].values
A = context['X_Action'].values
Y = context['Y_reward'].values

n_arms = 7

# randomly chosen by dice roll
n = 4569
# n = np.random.choice(context.index)

bandit = VetoBandit(X.shape[1], n_arms, step_size=0.01)

print('Probabilities for random row: ')
print(bandit.predict_proba(X[n]))

Probabilities for random row: 
[[0.16666667 0.16666667 0.16666667 0.16666667 0.         0.16666667
  0.16666667]]


In [31]:
X[n]

array([1, 1, 1, 1, 0, 1, 1])

In [32]:
n, A[n], Y[n]

(4569, 0, -1.0)

In [40]:
for match in context['MatchId'].unique():
    for team in context[context['MatchId'] == match]['DecisionTeamId'].unique():
        indices = (context['MatchId'] == match) & (context['DecisionTeamId'] == team)
        
        bandit.update_theta(X[indices], A[indices], Y[indices])

In [44]:
print('Probabilities for random row after training: ')
print(bandit.predict_proba(X[n+1]))
print('Selected action: ')
print(bandit.predict(X[n+1]))

print('Actual action and reward for random row: ')
print(f'Action: {A[n+1]}, reward: {Y[n+1]}')

Probabilities for random row after training: 
[[0.00000000e+00 0.00000000e+00 0.00000000e+00 9.80044400e-01
  0.00000000e+00 1.98981064e-02 5.74935356e-05]]
Selected action: 
[3]
Actual action and reward for random row: 
Action: 5, reward: -1.0
